In [ ]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import s3fs
from sqlalchemy import create_engine
import sys

In [2]:
load_dotenv()
folder_path = os.getcwd()

In [3]:
def engine_postgre(user, password, host, port, database):
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")
    return engine

In [4]:
postgres_user = os.getenv('POSTGRES_USER')
postgres_password = os.getenv('POSTGRES_PASSWORD')
postgres_host = os.getenv('POSTGRES_HOST')
postgres_host_local = os.getenv('POSTGRES_HOST_LOCAL')
postgres_port = os.getenv('POSTGRES_PORT')
postgres_port_local = os.getenv('POSTGRES_PORT_LOCAL')
postgres_database = os.getenv('POSTGRES_DB')

In [5]:
engine = engine_postgre(postgres_user, postgres_password, postgres_host, postgres_port, postgres_database)

In [ ]:
db_table = '"Exchange_Rates_Silver_Table"'

query = F""" 
SELECT MAX(DATE(time_last_update_utc)) FROM {db_table}
"""
last_update = pd.read_sql(query, engine).iloc[0, 0]
print(f'Last update: {last_update}')

2025-12-30


In [7]:
apy_key = os.getenv('APY_KEY')
url = f'https://v6.exchangerate-api.com/v6/{apy_key}/latest/USD'
response = requests.get(url)
data = response.json()
date_extract = pd.to_datetime(data['time_last_update_utc'])
file_date = date_extract.strftime('%Y-%m-%d')
df = pd.json_normalize(data)

In [ ]:
if date_extract <= last_update:
    print(f'Process Stopped: date_extract ({date_extract}) older or equal to last_update.')
    sys.exit(0)
else:
    print('Initializing loading process')

Iniciando proceso de carga


/tmp/ipykernel_48/3248339572.py:1: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if date_extract <= last_update:


In [9]:
# -------------------------------------------------------  RAW OBJECT STORE ---------------------------------------------#

In [10]:
minio_url = os.getenv('MINIO_BASE_URL')
minio_key = os.getenv('MINIO_KEY')
minio_secret = os.getenv('MINIO_SECRET')

In [11]:
fs = s3fs.S3FileSystem(key=minio_key, secret=minio_secret, client_kwargs={"endpoint_url": minio_url})
fs.ls("/")

['testdr']

In [12]:
bucket = 'testdr'
raw_key = f'{bucket}/bronze/{file_date}'
raw_path = f's3://{raw_key}.parquet'
raw_path

's3://testdr/bronze/2025-12-31.parquet'

In [13]:
try:
    df.to_parquet(raw_path, filesystem=fs, engine="pyarrow", index=False)
    print(f'File from date {file_date} loaded successfully')
except Exception as e:
    print(f'Error: {e}')

Archivo de fecha 2025-12-31 cargado exitosamente


In [14]:
silver_file = 'fullexchange_rates'
silver_key = f'{bucket}/silver/{silver_file}'
silver_path = f's3://{silver_key}.parquet'

In [ ]:
try:
    df_silver = pd.read_parquet(silver_path, filesystem=fs, engine="pyarrow")
    consolidated_df = pd.concat([df_silver, df], ignore_index=True)
    consolidated_df.to_parquet(silver_path, filesystem=fs, engine='pyarrow', index=False)
    print(f'DataFrame "{silver_file}" successfully loaded, total rows: {len(consolidated_df)}.')
except Exception as e:
    print(f'Error: {e}')

DataFrame "fullexchange_rates" combinado exitosamente, ahora tiene 41 filas.
